In [1]:
import nltk
import numpy as np
import urllib.request
import random
import string
import pyttsx3

from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords  

### Sintesis de Voz

In [2]:
engine = pyttsx3.init()
engine.setProperty('rate',150)
engine.setProperty('voice','spanish')

In [3]:
def speak(texto):
    engine.say(texto)
    engine.runAndWait()

In [4]:
#obtener el contenido HTML de la pagina web 
res = urllib.request.urlopen("https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/que-es-el-web-scraping/")
html = res.read()
soup = BeautifulSoup(html,"html.parser")

In [5]:
#observamos la etiquetas HTML
setTag = {x.name for x in soup.find_all(True)}
print(setTag)

{'a', 'link', 'article', 'meta', 'section', 'base', 'form', 'title', 'noscript', 'label', 'fieldset', 'h1', 'footer', 'time', 'i', 'picture', 'input', 'ol', 'nav', 'br', 'strong', 'body', 'h3', 'script', 'main', 'ul', 'head', 'style', 'p', 'img', 'em', 'div', 'span', 'button', 'source', 'li', 'header', 'h2', 'h4', 'html'}


In [6]:
def searchByTag(tagName):
    Content = ''
    for tag in soup.find_all(tagName):
        if(len(tag.text) > 144):
            Content += tag.text+'\n\n'
    return Content


In [7]:
Content = searchByTag('p')
# print(Content)

In [8]:
raw = Content.lower()

In [9]:
sent_tokens = nltk.sent_tokenize(raw) # convertir corpus a una lista de sentencias (tekenizar en sentencias)
word_tokens = nltk.word_tokenize(raw) # convertir corpus a una lista de palabras (tokenizar en palabras)

lemmer = nltk.stem.WordNetLemmatizer()#

In [10]:
remove_punct_dict = dict((ord(punct),None) for punct in string.punctuation) #remover signos de puntuacion (utilizamos la )

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] # Lematizar palabras tokenizadas

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [11]:
#Determinar la similutud del texto inserdado y el corpus

def response(user_response):# funcion respuesta (entrada: mensje de usuario)
    chatbot_response = '' #definir la respuesta del chatbot
    sent_tokens.append(user_response)# al listado de sentencias del corpus añadir al final de la lista el mensaje del usuario
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize,stop_words=stopwords.words('spanish'))# Lematizar y eliminar palbras
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # print("a",tfidf[-1])
    # print("------------------")
    # print(tfidf)
    # 3 Evaluar similitud coseno entre mensaje usuario (tfid[-1]) y el corpus(tfidf)
    vals = cosine_similarity(tfidf[-1],tfidf)
   

    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
  
    req_tfidf = flat[-2]
    # print("------------------")
    # print(req_tfidf)
    # print(idx)
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "Lo siento, no te he entendido"
        return chatbot_response
    
    else:
        chatbot_response = chatbot_response + sent_tokens[idx]
        return chatbot_response

In [12]:
SALUDOS_INPUTS = ("hola","buenas","saludos","que tal","hey","buenos dias")
SALUDOS_OUTPUTS = ["hola","hola, que tal?","Hola, Como te puedo ayudar?","hola, encantado de hablar contigo"]

def saludos(sentence):
    for word in sentence.split():
        # print(word.lower() in SALUDOS_INPUTS)
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

In [15]:
flag = True
introduces = "AGENTE: MI NOMBRE ES ROXANNE. CONTESTARE A SUS PREGUNTAS ACERCA DEL WEB SCRAPING Y SUS ASUNTOS LEGALES."

print(introduces)
speak(introduces)

while(flag == True):

    user_response = input()
    user_response = user_response.lower() #convertimos a minuscula

    if(user_response not in ["salir","chao","adios","bay","exit"]):

        if(user_response == 'gracias' or user_response == 'muchas gracias'): #se podria haber definido otra funcion de coincidencia
            flag = True
            thanks_res = "No hay de que"
            print("Roxanne:> "+ thanks_res)
            speak(thanks_res)
        else:
            if(saludos(user_response) != None): # si la palabra insertada por el usuario es un saludo (coincidencia manuales)
                greets = saludos(user_response) 
                print("Roxanne:> " + greets)
                speak(greets)

            else:#si la palabra insertada no es un saludo --> Corpus
                respond = response(user_response)
                print("Roxanne:> " + respond )
                speak(respond)
                sent_tokens.remove(user_response) # para eliminar del corpus la respues del usuario y volver a evaluar 
    else:
        flag = False
        print("Roxanne:> Nos vemos pronto, !Gracias por su visita¡")
        speak("Nos vemos pronto, !Gracias por su visita¡")    

AGENTE: MI NOMBRE ES ROXANNE. CONTESTARE A SUS PREGUNTAS ACERCA DEL WEB SCRAPING Y SUS ASUNTOS LEGALES.
Roxanne:> google es un buen ejemplo de web scraping.
Roxanne:> el scrapingno siempre es legal.
Roxanne:> los motores de búsqueda, como google, utilizan desde hace tiempo los denominados rastreadores web o crawlers, que exploran internet en busca de términos definidos por el usuario.
Roxanne:> No hay de que
Roxanne:> Nos vemos pronto, !Gracias por su visita¡


In [16]:
# print(Content)